In [1]:
# !/usr/bin/env python
# coding: utf-8

import warnings
warnings.simplefilter("ignore")
from training_functions import *

Se ha hecho uso de muchos de los procedimientos comentados en la competencia de modelos pero, aquí, han sido generalizados vía funciones contenidas en el script __training_functions.py__. Cada una de estas funciones tiene una descripción de la labor que llevan a cabo. 

En aquel script, la función __*optimization_model*__ realiza la optimización del modelo.

# 1. ESPACIO HIPERPARAMETRAL.

El `RandomForest`  se basa en la idea de construir múltiples árboles de decisión durante el entrenamiento y luego combinar sus resultados para obtener una predicción más precisa y robusta.

Se ha decidido trabajar con los siguientes hiperparámetros del estimador. 
1. __`n_estimators`:__  Número de árboles de decisión que se van a construir en el bosque aleatorio.
Mientras mayor sea, mejor ajuste aunque puede dar paso al sobreajuste.
5. __`max_depth`:__ Es la profundidad máxima permitida en los árboles. \
Ayuda a controlar la complejidad del modelo.
3. __`min_samples_leaf`:__ Número mínimo de muestras en cada hoja (nodos finales). \
Un valor alto puede evitar divisiones que sean demasiado específicas evitando a su vez el sobreajuste.
3. __`min_samples_split:__ Número mínimo de muestras requeridas para realizar una división en un nodo.
Ayuda a controlar la complejidad y el sobreajuste.
3. __`max_leaf_nodes`:__ Establece el número máximo de nodos hoja permitidos en cada árbol. 
Limitar la cantidad de hojas puede ayudar a evitar árboles muy profundos y complejos.

Por supuesto, hay más hiperparámetros que podemos controlar. Sin embargo, decidimos trabajar con estos últimos por ser los más representativos al momento de construir árboles de decisión.

El __espacio hiperparametral__ está conformado por todas las combinaciones de valores de hiperparámetros, dado un rango de acción para cada uno de ellos. A continuación se exhibe la configuración del espacio hiperparametral que se ha usado para la optimización.

In [2]:
space = {'random_state': [5000],
         'n_estimators': [100, 150, 200, 300, 600, 800],
         'max_depth': [None, 1],
         'min_samples_leaf': [1, 2, 3, 10, 20],
         'min_samples_split': [2, 3, 10, 30],
         'max_leaf_nodes': [None, 10, 70, 100, 150]}

La elección de esta configuración es meramente arbitraria, aunque la idea es considerar tanto valores altos como valores bajos de los hiperparámetros, para luego afinar los rangos de búsqueda en otras iteraciones de optimización.

Dado que el espacio hiperparámetral es limitado, la función __optimization_model__ utiliza un `GridSearchCV`, el cual recorrerá todo el espacio y aplicará un __cross-validation__ por cada punto en ese espacio. El total de puntos que recorrerá es de 1200. Si el espacio considerara rangos mucho más grandes de valores o bien considerara rangos continuos, entonces se debería utilizar un `RandomizedSearchCV` o un `BayesSearchCV`.

El sobreajuste del modelo se estará minitoriando al comparar las métricas de __cross-validation__ y las de __testeo__. Si éstas últimas sufren un deterioro importante, entonces habrá un serio problema de sobreajuste. Sin embargo, de acuerdo a experiencia propia, las métricas de __cross-validation__ suelen informar muy bien sobre la estabilidad general del modelo.

__Nota__: Aunque el modelo ha sido optimizado con el espacio hiperparametral anterior, en el script de funciones se ha dejado una versión más pequeña, esto para fines de prueba de código. Este espacio más reducido cuenta solo con 8 puntos de combinación.

# 2. CONFIGURACIONES GENERALES.

Los parámetros con controlan la función __optimization_model__ (muchos de ellos ya han sido explicados en la competencia de modelos) son:

In [4]:
# Localización de los archivos de salida.
path = '../outputs/'
# Son del binarios que contiene la data de entrenamiento.
df_name = 'df_HOMO_CLASS.sav'
# Lista de nombres de features y nombre de la variable objetivo.
features_names =  ['FLAG_OWN_CAR',
                   'FLAG_OWN_REALTY',
                   'NAME_INCOME_TYPE',
                   'NAME_EDUCATION_TYPE',
                   'NAME_FAMILY_STATUS',
                   'NAME_HOUSING_TYPE',
                   'OCCUPATION_TYPE',
                   'FLAG_WORK_PHONE',
                   'FLAG_PHONE',
                   'FLAG_EMAIL',
                   'AGE',
                   'CNT_CHILDREN',
                   'AMT_INCOME_TOTAL',
                   'WORK_YEARS']
objective_name = 'STATUS'
# Es el nombre del modelo ganador.
model_name = 'RandomForestClassifier'
# Es la semilla que gobernará los procesos.
seed = 5000
# Es la estrategia de submuestreo.
ratio_balance = 1
# Es el número de folds que conformarán el proceso de cross-validation.
k_folds = 4
# Un valor más grande hace más informativo el proceso de optimización.
verbose = 10
# Tamaño del conjunto de prueba.
test_size = 0.25
# Es la métrica de cross-validación que se optimizará. Por default es el f1.
# La combinación de hiperparámetros que obtenga el mayor valor de cv_f1 configurará el modelo optimizado final.
optimized_metric='cv_f1'
# Guarda en un binario la información del modelo optimizado.
save_best_info=False
# Ubicación de los binarios que contienen la data necesaria.
path='../outputs/'

# 3. OPTIMIZACIÓN Y GUARDADO DE INFORMACIÓN.

Dados los valores anteriores, procedemos a comenzar la optimización.

In [11]:
optim_results = optimization_model(df_name_sav=df_name,
                                   features_names=features_names,
                                   objective_name=objective_name,
                                   model_name=model_name,
                                   seed=seed,
                                   test_size=test_size,
                                   ratio_balance=ratio_balance,
                                   k_folds=k_folds,
                                   verbose=verbose,
                                   optimized_metric=optimized_metric,
                                   save_best_info=save_best_info,
                                   path=path)

2023-08-13 09:25:35 INFO: PROCESO DE OPTIMIZACIÓN.
2023-08-13 09:25:35 INFO: CONFIGURACIÓN GENERALES.
2023-08-13 09:25:35 INFO: TUBERÍAS PARA PROCESAMIENTO Y MUESTREO.
2023-08-13 09:25:35 INFO: CONFIGURACIÓN PARA CV Y SEARCH.
2023-08-13 09:25:35 INFO: COMIENZA OPTIMIZACIÓN.


Fitting 4 folds for each of 1200 candidates, totalling 4800 fits
[CV 1/4; 1/1200] START estimator__max_depth=None, estimator__max_leaf_nodes=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100, estimator__random_state=5000
[CV 1/4; 1/1200] END estimator__max_depth=None, estimator__max_leaf_nodes=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100, estimator__random_state=5000; f1: (test=0.261) f1_micro: (test=0.624) m_c: (test=0.171) precision: (test=0.163) precision_micro: (test=0.624) recall: (test=0.660) recall_micro: (test=0.624) total time=   0.2s
[CV 2/4; 1/1200] START estimator__max_depth=None, estimator__max_leaf_nodes=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100, estimator__random_state=5000
[CV 2/4; 1/1200] END estimator__max_depth=None, estimator__max_leaf_nodes=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estim

[CV 3/4; 4/1200] END estimator__max_depth=None, estimator__max_leaf_nodes=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=300, estimator__random_state=5000; f1: (test=0.249) f1_micro: (test=0.632) m_c: (test=0.152) precision: (test=0.157) precision_micro: (test=0.632) recall: (test=0.613) recall_micro: (test=0.632) total time=   0.5s
[CV 4/4; 4/1200] START estimator__max_depth=None, estimator__max_leaf_nodes=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=300, estimator__random_state=5000
[CV 4/4; 4/1200] END estimator__max_depth=None, estimator__max_leaf_nodes=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=300, estimator__random_state=5000; f1: (test=0.247) f1_micro: (test=0.653) m_c: (test=0.145) precision: (test=0.158) precision_micro: (test=0.653) recall: (test=0.570) recall_micro: (test=0.653) total time=   0.6s
[CV 1/4; 5/1200] START estimator__m

[CV 2/4; 8/1200] END estimator__max_depth=None, estimator__max_leaf_nodes=None, estimator__min_samples_leaf=1, estimator__min_samples_split=3, estimator__n_estimators=150, estimator__random_state=5000; f1: (test=0.224) f1_micro: (test=0.585) m_c: (test=0.112) precision: (test=0.138) precision_micro: (test=0.585) recall: (test=0.602) recall_micro: (test=0.585) total time=   0.3s
[CV 3/4; 8/1200] START estimator__max_depth=None, estimator__max_leaf_nodes=None, estimator__min_samples_leaf=1, estimator__min_samples_split=3, estimator__n_estimators=150, estimator__random_state=5000
[CV 3/4; 8/1200] END estimator__max_depth=None, estimator__max_leaf_nodes=None, estimator__min_samples_leaf=1, estimator__min_samples_split=3, estimator__n_estimators=150, estimator__random_state=5000; f1: (test=0.245) f1_micro: (test=0.644) m_c: (test=0.143) precision: (test=0.156) precision_micro: (test=0.644) recall: (test=0.581) recall_micro: (test=0.644) total time=   0.3s
[CV 4/4; 8/1200] START estimator__m

[CV 1/4; 12/1200] END estimator__max_depth=None, estimator__max_leaf_nodes=None, estimator__min_samples_leaf=1, estimator__min_samples_split=3, estimator__n_estimators=800, estimator__random_state=5000; f1: (test=0.270) f1_micro: (test=0.635) m_c: (test=0.185) precision: (test=0.169) precision_micro: (test=0.635) recall: (test=0.670) recall_micro: (test=0.635) total time=   1.2s
[CV 2/4; 12/1200] START estimator__max_depth=None, estimator__max_leaf_nodes=None, estimator__min_samples_leaf=1, estimator__min_samples_split=3, estimator__n_estimators=800, estimator__random_state=5000
[CV 2/4; 12/1200] END estimator__max_depth=None, estimator__max_leaf_nodes=None, estimator__min_samples_leaf=1, estimator__min_samples_split=3, estimator__n_estimators=800, estimator__random_state=5000; f1: (test=0.230) f1_micro: (test=0.599) m_c: (test=0.122) precision: (test=0.142) precision_micro: (test=0.599) recall: (test=0.602) recall_micro: (test=0.599) total time=   1.1s
[CV 3/4; 12/1200] START estimato

...

[CV 1/4; 1191/1200] END estimator__max_depth=1, estimator__max_leaf_nodes=150, estimator__min_samples_leaf=20, estimator__min_samples_split=10, estimator__n_estimators=200, estimator__random_state=5000; f1: (test=0.189) f1_micro: (test=0.531) m_c: (test=0.043) precision: (test=0.114) precision_micro: (test=0.531) recall: (test=0.543) recall_micro: (test=0.531) total time=   0.3s
[CV 2/4; 1191/1200] START estimator__max_depth=1, estimator__max_leaf_nodes=150, estimator__min_samples_leaf=20, estimator__min_samples_split=10, estimator__n_estimators=200, estimator__random_state=5000
[CV 2/4; 1191/1200] END estimator__max_depth=1, estimator__max_leaf_nodes=150, estimator__min_samples_leaf=20, estimator__min_samples_split=10, estimator__n_estimators=200, estimator__random_state=5000; f1: (test=0.169) f1_micro: (test=0.526) m_c: (test=0.009) precision: (test=0.103) precision_micro: (test=0.526) recall: (test=0.484) recall_micro: (test=0.526) total time=   0.3s
[CV 3/4; 1191/1200] START estima

[CV 3/4; 1194/1200] END estimator__max_depth=1, estimator__max_leaf_nodes=150, estimator__min_samples_leaf=20, estimator__min_samples_split=10, estimator__n_estimators=800, estimator__random_state=5000; f1: (test=0.225) f1_micro: (test=0.586) m_c: (test=0.113) precision: (test=0.138) precision_micro: (test=0.586) recall: (test=0.602) recall_micro: (test=0.586) total time=   0.9s
[CV 4/4; 1194/1200] START estimator__max_depth=1, estimator__max_leaf_nodes=150, estimator__min_samples_leaf=20, estimator__min_samples_split=10, estimator__n_estimators=800, estimator__random_state=5000
[CV 4/4; 1194/1200] END estimator__max_depth=1, estimator__max_leaf_nodes=150, estimator__min_samples_leaf=20, estimator__min_samples_split=10, estimator__n_estimators=800, estimator__random_state=5000; f1: (test=0.189) f1_micro: (test=0.494) m_c: (test=0.044) precision: (test=0.112) precision_micro: (test=0.494) recall: (test=0.591) recall_micro: (test=0.494) total time=   1.0s
[CV 1/4; 1195/1200] START estima

[CV 1/4; 1198/1200] END estimator__max_depth=1, estimator__max_leaf_nodes=150, estimator__min_samples_leaf=20, estimator__min_samples_split=30, estimator__n_estimators=300, estimator__random_state=5000; f1: (test=0.185) f1_micro: (test=0.547) m_c: (test=0.037) precision: (test=0.113) precision_micro: (test=0.547) recall: (test=0.511) recall_micro: (test=0.547) total time=   0.5s
[CV 2/4; 1198/1200] START estimator__max_depth=1, estimator__max_leaf_nodes=150, estimator__min_samples_leaf=20, estimator__min_samples_split=30, estimator__n_estimators=300, estimator__random_state=5000
[CV 2/4; 1198/1200] END estimator__max_depth=1, estimator__max_leaf_nodes=150, estimator__min_samples_leaf=20, estimator__min_samples_split=30, estimator__n_estimators=300, estimator__random_state=5000; f1: (test=0.176) f1_micro: (test=0.538) m_c: (test=0.022) precision: (test=0.107) precision_micro: (test=0.538) recall: (test=0.495) recall_micro: (test=0.538) total time=   0.5s
[CV 3/4; 1198/1200] START estima

2023-08-13 10:08:35 INFO: CÁLCULO DE MÉTRICAS DE CV.
2023-08-13 10:08:35 INFO: SELECCIÓN DEL MEJOR MODELO VÍA UNA MÉTRICA DE CV.
2023-08-13 10:08:35 INFO: FIN DE OPTIMIZACIÓN.


[CV 4/4; 1200/1200] END estimator__max_depth=1, estimator__max_leaf_nodes=150, estimator__min_samples_leaf=20, estimator__min_samples_split=30, estimator__n_estimators=800, estimator__random_state=5000; f1: (test=0.189) f1_micro: (test=0.494) m_c: (test=0.044) precision: (test=0.112) precision_micro: (test=0.494) recall: (test=0.591) recall_micro: (test=0.494) total time=   1.1s


In [12]:
optim_results

hyperparameters       {'max_depth': None, 'max_leaf_nodes': None, 'm...
cv_f1                                                          0.250567
cv_precision                                                   0.156689
cv_recall                                                      0.627231
cv_m_c                                                         0.153587
cv_f1_micro                                                    0.623491
cv_precision_micro                                             0.623491
cv_recall_micro                                                0.623491
optimization_date                            2023-08-13 10:08:35.636502
ratio_balance                                                         1
model_name                                       RandomForestClassifier
k_folds                                                               4
test_size                                                          0.25
Name: 0, dtype: object

Al final, la función __optimization_model__ devuelve un panda-series con la información del modelo optimizado.

Se guardan la información del modelo optimizado.

In [13]:
# pickle.dump(optim_results, open('../outputs/optimization_results.sav', 'wb'))

# 4. COMPARACIÓN ENTRE EL MODELO OPTIMIZADO Y SIN OPTIMIZAR.

In [14]:
optim_results = get_data(name_sav='optimization_results.sav',
                         path=path)

La combinación ganadora de hiperparámetros es la siguiente.

In [15]:
optim_results['hyperparameters']

{'max_depth': None,
 'max_leaf_nodes': None,
 'min_samples_leaf': 3,
 'min_samples_split': 3,
 'n_estimators': 800,
 'random_state': 5000}

A simple vista, observamos que el modelo está requiriendo muchos árboles (__`max_iter`__=800) para poder mejorar en su rendimiento.

Comparemos las métricas de __cross-validation__ del modelo optimizado con las del modelo no optimizado (ver __model_selection.ipynb__)

In [17]:
print('f1-cv optimizado', round(optim_results['cv_f1'], 2))
print('f1-cv NO optimizado', 0.24)
print('recall-cv optimizado', round(optim_results['cv_recall'], 2))
print('recall-cv NO optimizado',  0.6)
print('precision-cv optimizado', round(optim_results['cv_precision'], 2))
print('precision-cv NO optimizado', 0.15)
print('m_c-cv optimizado', round(optim_results['cv_m_c'], 2))
print('m_c-cv NO optimizado', 0.14)   

f1-cv optimizado 0.25
f1-cv NO optimizado 0.24
recall-cv optimizado 0.63
recall-cv NO optimizado 0.6
precision-cv optimizado 0.16
precision-cv NO optimizado 0.15
m_c-cv optimizado 0.15
m_c-cv NO optimizado 0.14


Vemos que solo se han conseguido mejoras de a lo más dos centésimas en todos las métricas de __cross-validation__.

Habrá que hacer un revisión desde el principio, con tal de encontrar mejoras contundentes. Los enfoques que se podrían llevar a cabo son:
1. Enriquecer 
2. Hacer una ingeniería de variables más ambiciosa y rica. Aprovechar de manera más ingeniosa la variable de tiempo, por ejemplo.
3. Cambiar el estimador. El gran tamaño de la data nos da acceso a herramientas más complejas como redes neuronales.
4. Utilizar métodos de ensamblado de modelos.
5. Considerar otro enfoque de muestreo. Tal vez considerar técnicas como el SMOTE u otra ténica de sobremuestreo de la clase minoritaria.